In [ ]:
import easyquotation
import decimal
import pendulum
import math
import requests
import easyutils
import redis
import time
import platform
import subprocess
import types
import os
import struct
import pandas as pd
import numpy as np

from random import randrange
from pytdx.hq import TdxHq_API
from pytdx.params import TDXParams
from pytdx.config.hosts import hq_hosts as tdx_hq_hosts
from pytdx.reader import TdxDailyBarReader, TdxFileNotFoundException

In [ ]:
decimal.getcontext().rounding = decimal.ROUND_HALF_UP

In [ ]:
class HQ:
    def __init__(self, tdx_dir=None):
        
        self.redis = redis.Redis(host='127.0.0.1', port=6379, decode_responses=True)
        
        self.tdx = TdxHq_API(heartbeat=True, auto_retry=True)
        self.tdx_connect_to_server()
        self.tdx_dir = tdx_dir
        self.tdx_hq_hosts = tdx_hq_hosts
        
        self.sina = easyquotation.use('sina')
        self.timekline = easyquotation.use('timekline')
        self.snapshot = self.sina.market_snapshot(prefix=False)
        
        self.last_trading_date = self.get_last_trading_date()
        
        self.update_stock_codes()
        
        self.codes = self.get_stock_codes()
        
        
    def get_last_trading_date(self):
        stock = self.sina.real('399001')['399001']
        dt_str = stock['date']+' '+stock['time']
        return pendulum.parse(dt_str, tz='Asia/Shanghai')
    
    
    def get_zt_stocks(self, trading_date=None):

        if not trading_date or not self.is_trading_day(trading_date):
            trading_date = self.last_trading_date

        stocks = None

        if trading_date.to_date_string() == self.last_trading_date.to_date_string():
            stocks = self.snapshot
        elif self.is_trading_day(trading_date):
            stocks = self.get_historic_snapshot(trading_date)

        zt_stocks = {}
        for code in stocks:

            stock = stocks[code]

            if stock['close'] == 0:
                continue

            if stock['date'] != trading_date.to_date_string():
                continue

            high = stock['high']
            now = stock['now']
            close = stock['close']
            zt_price = float(round(decimal.Decimal(close * 1.1), 2))

            stock['zt_price'] = zt_price

            if high >= zt_price or high/close >=1.095:

                if now >= zt_price:
                    stock['is_zt'] = True
                    stock['is_lanban'] = False
                elif high >= zt_price:
                    stock['is_zt'] = False
                    stock['is_lanban'] = True
                else:
                    stock['is_zt'] = False
                    stock['is_lanban'] = False

                zt_stocks[code] = stock

        data_fs = {}
        for i in range(20):
            codes = list(set(list(zt_stocks.keys())) - set(list(data_fs.keys())))
            if not codes:
                break

            result = self.timekline.real(codes, prefix=True)
            for key in result:
                code = key[2:-3]
                data_fs[code] = result[key]

        assert len(zt_stocks) == len(data_fs)

        for code in zt_stocks:
            stock = zt_stocks[code]
            fs = data_fs[code]

            assert trading_date.format('YYYYMMDD') == fs['date']

            time_mp = None
            high = 0.00
            for data in fs['time_data']:
                time, price, vol = data
                price = float(price)
                if high < price:
                    high = price
                    time = time[:2] + ':' + time[2:] + ":00"
                    stock['peek_time'] = trading_date.to_date_string() + ' ' + time

        zt_stocks = list(zt_stocks.values())
        zt_stocks.sort(key=lambda x: x['peek_time'])

        return zt_stocks
    
    def get_historic_snapshot(self, trading_date):
        
        # 检测 通达信 本地数据 是否 满足本次调用
        assert self.is_tdx_local_data_ready_for(trading_date)
        
        reader = TdxDailyBarReader()

        date_str = trading_date.to_date_string()
        sh = self.tdx_dir+'\\vipdoc\\sh\\lday\\'
        sz = self.tdx_dir+'\\vipdoc\\sz\\lday\\'

        stocks = {}
        codes = self.get_stock_codes()

        total = len(codes)

        for idx, code in enumerate(codes):


            if code.startswith('6'):
                file = sh + 'sh' + code +'.day'
            else:
                file = sz + 'sz' + code +'.day'

            print(str(idx+1)+'/'+str(total)+', '+file+' ', end='\r')

            if not os.path.exists(file):
                continue

            df = reader.get_df(file)

            if date_str in df.index:
                stock = df.loc[date_str].to_dict()
                stock['date'] = date_str
                stock['now'] = stock['close']

                index = df.index.get_loc(date_str)
                if index == 0:
                    stock['close'] = stock['open']
                else:
                    index -= 1
                    stock['close'] = df.iloc[index].to_dict()['close']

                stocks[code] = stock

        return stocks
    
    def get_stock_codes(self):
        return easyquotation.get_stock_codes()
    
    def is_trading_day(self, date):
        url = 'http://tool.bitefu.net/jiari/?d='+date.format('YYYYMMDD')
        if int(requests.get(url).content) == 0:
            return True
        else:
            return False
        # return not easyutils.is_holiday(date.format('YYYYMMDD'))
    
    def is_stock_codes_update_needed(self):
        update_dt = self.redis.get('stock_codes_update_time')
        if not update_dt:
            return True
        timezone = 'Asia/Shanghai'
        update_dt = pendulum.parse(update_dt, tz=timezone)

        dt = pendulum.now()
        if not self.is_trading_day(dt):
            dt = self.last_trading_date

        compare_dt = pendulum.parse(dt.to_date_string() + ' 09:00:00', tz=timezone)

        if dt.to_date_string() == pendulum.now().to_date_string() and dt <= compare_dt:
            return False
        
        if update_dt <= compare_dt:
            return True
        else:
            return False

    def update_stock_codes(self, force=False):
        
        # if not self.is_tdx_local_data_ready_for(self.last_trading_date):
        #     print('通达信 本地数据 需要更新！')
        #     return
        
        old_codes = easyquotation.get_stock_codes()
        is_update_needed = self.is_stock_codes_update_needed()

        if is_update_needed or force:
            if is_update_needed:
                print('更新股票代码')
            code_filter = []
            code_filter.append(['000', '002', '300'])
            code_filter.append(['600', '601', '603', '688'])


            step_size = 1000
            codes = []
            for market in range(2):

                count = self.tdx.get_security_count(market)
                print(market, count, end=' : ')

                steps = math.ceil(count/step_size)

                total = 0
                for step in range(steps):
                    result = self.tdx.get_security_list(market, step_size*step)
                    print(str(step)+'/'+str(steps), end=", ")
                    for item in result:
                        code = item['code'].strip()
                        if code[:3] in code_filter[market]:
                            codes.append(code)
                            total += 1

                print('summe:'+str(total))
            print('summe:'+str(len(codes)))

            easyquotation.update_stock_codes(codes)
            self.sina.codes = codes
            
            print('新增 股票 代码：', set(codes)-set(old_codes))

            self.redis.set('stock_codes_update_time', pendulum.now().to_datetime_string())
        else:
            print('股票代码已更新')
            
    def test_tdx_hosts(self):
        hosts = []
        for host in tdx_hq_hosts:
            print('ping host:', host, end=", ")
            time = self.ping(host[1])
            print(time, 'ms')
            host = [*host]
            host.append(time)
            hosts.append(host)

        hosts.sort(key=lambda x : x[3])

        key = 'tdx_hosts'
        self.redis.delete(key)
        for host in hosts:
            if host[3] > 100:
                break
            self.redis.rpush(key, host[1]+':'+str(host[2]))
        return hosts
    
    def tdx_connect_to_server(self):
        key = 'tdx_hosts'
        
        if self.redis.exists("tdx_hosts") == 0:
            print('use default server: ', '202.108.253.130:7709')
            print('please run test_tdx_hosts to test out fast servers')
            self.tdx.connect('202.108.253.130', 7709)
        else:
            idx = -1
            min_host = None
            min_time = None
            while True:
                idx += 1
                host = self.redis.lindex(key, idx)
                if host is None:
                    break

                host = host.split(':')
                print(host, end=", ")
                try:
                    time = self.ping(host[0])
                except Exception as e:
                    continue
                print('used', round(time,2), 'ms')

                if min_time is None or min_time > time:
                    min_time = time
                    min_host = (host[0], int(host[1]))


            print('selected:', min_host, round(min_time, 2), 'ms')
            self.tdx.connect(*min_host)
            
    def get_tdx_gainian(self):
        assert self.tdx_dir

        fname = self.tdx_dir + '\\T0002\hq_cache\\block_gn.dat'
        result = {}
        if type(fname) is not bytearray:
            with open(fname, "rb") as f:
                data = f.read()
        else:
            data = fname

        pos = 384
        (num, ) = struct.unpack("<H", data[pos: pos + 2])
        pos += 2
        for i in range(num):
            blockname_raw = data[pos: pos + 9]
            pos += 9
            name = blockname_raw.decode("gbk", 'ignore').rstrip("\x00")
            stock_count, block_type = struct.unpack("<HH", data[pos: pos + 4])
            pos += 4
            block_stock_begin = pos
            codes = []
            for code_index in range(stock_count):
                one_code = data[pos: pos + 7].decode("utf-8", 'ignore').rstrip("\x00")
                codes.append(one_code)
                pos += 7

            gn = {}
            gn["name"] = name
            gn["block_type"] = block_type
            gn["stock_count"] = stock_count
            gn["codes"] = codes
            result[name] = gn

            pos = block_stock_begin + 2800

        return result
    
    def get_tdx_hangye(self):
        assert self.tdx_dir

        file_hangye = self.tdx_dir + '\\incon.dat'
        assert os.path.exists(file_hangye)
        file_stock_hangye = self.tdx_dir + '\T0002\hq_cache\\tdxhy.cfg'
        assert os.path.exists(file_stock_hangye)

        result = {}
        with open(file_hangye, "rt", encoding='gb2312') as f:
            isTDXHY = False
            for line in f:
                line = line.rstrip()
                if not isTDXHY and line != '#TDXNHY':
                    continue
                elif not isTDXHY and line == '#TDXNHY':
                    isTDXHY = True
                    continue
                elif isTDXHY and line == '######':
                    isTDXHY = False
                    break
                code, name = line.split('|')
                result[code] = {}
                result[code]['code'] = code
                result[code]['name'] = name
                result[code]['codes'] = []

        with open(file_stock_hangye, "rt", encoding='gb2312') as f:
            for line in f:
                line = line.rstrip()
                market_code, stock_code, tdxhy_code, swhy_code, unknown_code = line.split("|")
                stock_code = stock_code.strip()

                if tdxhy_code != 'T00':
                    result[tdxhy_code]['codes'].append(stock_code)
        return result
    
    def get_tdx_zhishu(self):
        assert self.tdx_dir

        tdxzs_cfg = self.tdx_dir + '\\T0002\\hq_cache\\tdxzs.cfg'
        gainian = self.get_tdx_gainian()
        hangye = self.get_tdx_hangye()

        result = {}
        with open(tdxzs_cfg, "rt", encoding='gb2312') as f:
            for line in f:
                line = line.rstrip()
                zs_name, zs_code, zs_type, num_1, num_2, key = line.split('|')

                if key in gainian:
                    if zs_code in result:
                        print('------------------------------------------------------')
                        print('in result key: ', key, zs_name, zs_code)
                        print('gainian: ', key, gainian[key])
                        continue
                    else:
                        if len(gainian[key]['codes']) == 0:
                            continue
                        zs = {}
                        zs['code'] = zs_code
                        zs['name'] = gainian[key]['name']
                        zs['codes'] = gainian[key]['codes']
                        result[zs_code] = zs

                if key in hangye:
                    if zs_code in result:
                        print('------------------------------------------------------')
                        print('in result key: ', key, zs_name, zs_code)
                        print('hangye: ', key, hangye[key])
                        continue
                    else:
                        if len(hangye[key]['codes']) == 0:
                            continue
                        zs = {}
                        zs['code'] = zs_code
                        zs['name'] = hangye[key]['name']
                        zs['codes'] = hangye[key]['codes']
                        result[zs_code] = zs

        return result

    def is_tdx_local_data_ready_for(self, date):
        file = self.tdx_dir+'\\vipdoc\\sz\\lday\\sz399001.day'
        reader = TdxDailyBarReader()
        df = reader.get_df(file)

        return date.to_date_string() in df.index
    
    def ping(self, host):
    
        param = '-n' if platform.system().lower()=='windows' else '-c'
        command = ['ping', param, '1', host]

        start = time.time()
        subprocess.call(command)
        end = time.time()
        return (end - start)*1000

In [ ]:
tdx_dir = 'C:\\tdx\\tdx_7.46'
hq = HQ(tdx_dir=tdx_dir)

In [ ]:
hq.update_stock_codes(force=True)

In [ ]:
# hq.update_stock_codes = types.MethodType(update_stock_codes, hq)

In [ ]:
codes = hq.get_stock_codes()
len(codes)

In [ ]:
ms = hq.sina.market_snapshot(prefix=False)
len(ms)

In [ ]:
view = np.empty(len(ms), 7, dtype=np.float32)

for idx, code in enumerate(codes):
    stock = ss[code]
    view[idx, 0] = stock['open'] if stock['open'] != 0 else np.nan
    view[idx, 1] = stock['close'] if stock['close'] != 0 else np.nan
    view[idx, 2] = stock['now'] if stock['now'] != 0 else np.nan
    view[idx, 3] = stock['high'] if stock['high'] != 0 else np.nan
    view[idx, 4] = stock['low'] if stock['low'] != 0 else np.nan
    view[idx, 5] = stock['turnover'] if stock['turnover'] != 0 else np.nan
    view[idx, 6] = stock['volume'] if stock['volume'] != 0 else np.nan


In [ ]:
%%timeit -n10 -c
view = np.empty((len(ms), 7), dtype=np.float32)
view.fill(np.nan)

for idx, code in enumerate(codes):
    stock = ms[code]
    view[idx, 0] = stock['open'] if stock['open'] != 0 else np.nan
    view[idx, 1] = stock['close'] if stock['close'] != 0 else np.nan
    view[idx, 2] = stock['now'] if stock['now'] != 0 else np.nan
    view[idx, 3] = stock['high'] if stock['high'] != 0 else np.nan
    view[idx, 4] = stock['low'] if stock['low'] != 0 else np.nan
    view[idx, 5] = stock['turnover'] if stock['turnover'] != 0 else np.nan
    view[idx, 6] = stock['volume'] if stock['volume'] != 0 else np.nan

In [ ]:
l1 = hq.tdx.get_history_minute_time_data(TDXParams.MARKET_SH, '600300', 20190718)
l2 = hq.timekline.real('600300')

In [ ]:
l = []
lv = 0
for idx, (t, p, v) in enumerate(l2['sh600300.js']['time_data']):
    l.append([t,p,int(v)-lv])
    lv = int(v)

In [ ]:
tdx = TdxHq_API(heartbeat=True, auto_retry=True)

In [ ]:
tdx.get_company_info_category(TDXParams.MARKET_SZ, '002717')

In [ ]:
import tushare as TS

In [ ]:
ts = TS.pro_api('aecca28adc0d5a7764b748ccd48bef923d81314ae47b4d44d51fce67')

In [ ]:
set(codes)-set(ts.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')['symbol'])

In [ ]:
ts.trade_cal(exchange='', start_date='20190726', end_date='20190731')

In [ ]:
ts.daily_basic(ts_code='', trade_date='20190726', fields='ts_code,total_share,float_share,free_share')

In [ ]:
import redis

In [ ]:
r = redis.Redis(host='127.0.0.1', port=6379, decode_responses=True)

In [ ]:
not r.get('dkjlfs') 

In [1]:
from libs.utils import Utils
import easyquotation

In [ ]:
codes = easyquotation.get_stock_codes()
len(codes)

In [ ]:
Utils.save_stock_codes(codes)

In [ ]:
codes = Utils.get_stock_codes()
len(codes)

In [2]:
Utils.is_stock_codes_update_needed()

True